# Press the double arrow above and enjoy! This program allows you to examine gene expression in all cell clusters identified in Loo, L., Simon, J. M., Xing, L., McCoy, E. S., Niehaus, J. K., Guo, J., ... & Zylka, M. J. (2019). Single-cell transcriptomic analysis of mouse neocortical development. Nature communications, 10(1), 1-11.



# Cells are grouped into Cell Types (Inhibitory, Cortical Layers, Non-neuronal, Proliferative, Midbrain). They are further divided into Clusters (identified by Loo et al) and age of sample. Each 'block' on each bar is an individual cluster. 

In [1]:
import matplotlib 
import ipywidgets as widgets
import pandas as pd
import io
import seaborn as sns
from IPython.display import display
from ipywidgets import interact_manual, interactive,GridspecLayout, AppLayout, Button, Layout, Widget
import plotly.express as px

data = pd.read_csv("LooSimon_MouseCortexDropSeq_E14P0_avgExpByCellType.csv")
# Get column names 
listOfColumnNames = list(data.columns.values[1:])
idx = list(data['GeneID'])

In [2]:
cells =[]
genes = []

def on_go_button_clicked(go_button):

#Mark errors
    if len(genes) < 1:
        with outerrors:
            print("Choose at least one Gene")
    if len(list(Cells_drop.value)) < 1:
        with outerrors:
            print("Choose at least one Cell Cluster")
            
## restructure data frame to include selected types only 
    if len(genes) >= 1:
         if len(list(Cells_drop.value)) >= 1:
            data_gene_cell = data[list(Cells_drop.value)]
            data_gene_cell.index= data['GeneID']
            data_gene_cell = data_gene_cell.loc[genes]
            data_gene_cell = data_gene_cell.reset_index()

        ## reshape for plotting
            data_gene_cell2 = pd.melt(data_gene_cell, id_vars =['GeneID'],
                              var_name ='Cluster', value_name ='Expression')

        ## plot
            ages = []
            cell_type = []
            supergroup = []
            import re

            for i in data_gene_cell2['Cluster']:    
                if '-P]' in i:
                    ages.append('p0')
                if '-E]' in i:
                    ages.append('e14.5')
                if 'Layer' in i:
                    supergroup.append("Cortical Layers")
                if any(x in i for x in ['Endothelial','Choroid','Astrocyte','Oligo','Ganglionic','Micro']): 
                    supergroup.append("Non-Neuronal")
                if any(x in i for x in ['SVZ','RG']): 
                    supergroup.append("Proliferative")
                if "Int" in i:
                    supergroup.append("Inhibitory")
                if any(x in i for x in ['Thalamic','Striatal']): 
                    supergroup.append("Midbrain")
                j = re.sub("[\[].*?[\]]", "", i)
                cell_type.append(j)

            data_gene_cell2['Cell_Type'] = cell_type
            data_gene_cell2['Age'] = ages
            data_gene_cell2['Category'] = supergroup


            import plotly.graph_objects as go
            fig = px.bar(data_gene_cell2, x="Category", y="Expression", color = "Age", 
                    facet_col = "GeneID",
                    hover_name = "Cell_Type",
                    hover_data = ["Age", "Cluster"]
                    )
            fig.update_layout(barmode='group')
            with graph_out1:
                    fig.show()

            ## print table
            fig2 = go.Figure(data=[go.Table(
            header=dict(values=list(data_gene_cell2.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[data_gene_cell2.GeneID, data_gene_cell2.Cluster, data_gene_cell2.Expression,
                              data_gene_cell2.Cell_Type, data_gene_cell2.Age, data_gene_cell2.Category],
                    fill_color='lavender',
                    align='left'))
                ])
            with graph_out2:
                    fig2.show()


def choose_gene(change):
    if dropdown.value not in genes:
        if dropdown.value != '':
            genes.append(dropdown.value)
            with outgenes:
                display(dropdown.value)

In [3]:
##define GUI  

spacer = widgets.Label('')
genetitle = widgets.Label('Select Genes')

dropdown =  widgets.Combobox(
    options= idx,
    description='Gene',
    placeholder='Type here',
    layout={'width': '80%'})

   
outgenes = widgets.Output()   
outgraph = widgets.Output(layout={'border': '1px solid black'})  

        
gene_button = widgets.Button(description = 'Enter Gene', 
                button_style = '', 
                display='flex',
                flex_flow='column',
                align_items='stretch')

gene_button.on_click(choose_gene)

celllabel = widgets.Label('Cell Clusters:')
Cells_drop =widgets.SelectMultiple(
    options=listOfColumnNames,
    rows=8,
    layout={'width': 'max-content'}) 


go_button = widgets.Button(description = 'Produce Output', 
                           button_style = '', 
                            display='flex',
                            flex_flow='column',
                            align_items='stretch')
go_button.on_click(on_go_button_clicked)

gene_button.observe(dropdown.value)
go_button.observe(dropdown.value)
go_button.observe(Cells_drop.value)


graph_out1 = widgets.Output()  
graph_out2 = widgets.Output()  

In [ ]:
genesbox = widgets.VBox([genetitle,dropdown, gene_button])
cellsbox = widgets.VBox([celllabel, Cells_drop])
genechoices = widgets.VBox([genesbox, outgenes,go_button])
outerrors = widgets.Output() 

graphit = widgets.VBox([graph_out1, graph_out2])
top_app = widgets.HBox([cellsbox,genechoices, outerrors])
app = widgets.VBox([top_app, graphit])


app